In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
import json

# Dataset path
dataset_dir = r"D:\cattle"

#  Parameters
img_size = (224, 224)
batch_size = 32
validation_split = 0.2  # 20% for validation
seed = 123
AUTOTUNE = tf.data.AUTOTUNE

#  Load training dataset from same directory with split
train_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "Buffalo"),
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="int",
    shuffle=True
)

#  Load validation dataset from same directory with split
val_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "Buffalo"),
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="int",
    shuffle=True
)

#  Get number of classes
num_classes = len(train_ds.class_names)
print("Detected classes:", train_ds.class_names)

#  Optional: Save class names for later use
with open("breed_class_names.json", "w") as f:
    json.dump(train_ds.class_names, f)

#  Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

#  Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="data_augmentation")

#  Load EfficientNetB0 as base model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base model

#  Build model
inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

#  Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#  Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=200
)

#  Save model in both formats
model.save("Buffalo_breed.keras")  # Recommended format
model.save("Buffalo_breed.h5")     # Legacy format

print(" Model saved successfully.")


Found 357 files belonging to 5 classes.
Using 286 files for training.
Found 357 files belonging to 5 classes.
Using 71 files for validation.
Detected classes: ['Banni', 'Jaffrabadi', 'Nagpuri', 'Nili_ravi', 'Toda']
Epoch 1/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.1569 - loss: 1.8327 - val_accuracy: 0.2535 - val_loss: 1.6375
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2623 - loss: 1.6529 - val_accuracy: 0.2676 - val_loss: 1.5905
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2940 - loss: 1.6158 - val_accuracy: 0.3380 - val_loss: 1.5518
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2498 - loss: 1.6267 - val_accuracy: 0.3803 - val_loss: 1.5179
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.3073 - loss: 1.5229 - val_accuracy: 0.3803 - val_loss: 1.4878
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.3106 - loss: 1.5089 - val_accuracy: 0.4085 - val_loss: 1.4644
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━

 Model saved successfully.


In [ ]:
how to use the model

In [2]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import os

# Define your class names in the same order used during training
class_names = ["Banni" ,"Jaffrabadi" , "Nagpuri" , "Nili_ravi" ,"Toda"] # <-- update to match your dataset

def predict_image(path, model):
    img = image.load_img(path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    # img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)    //////  this convertion loses the data and give wrong pridiction
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    print(f"Prediction scores: {predictions}")
    return predicted_class

# Load model
model = tf.keras.models.load_model("Cow_breed.keras")

# Predict
print(predict_image("nagpuri.png", model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction scores: [[0.02366685 0.54040295 0.12965055 0.08797994 0.21829973]]
Jaffrabadi
